# YAPILACAKLAR:
* Check min(boundary iou, iou) loss function implementation
* test time augmentation (tta)
* reduce rl on plateo: load best ckpt model when reducing learning rate. Callback bunu yapıyor. Fakat best ckpt model yüklenince patience/learning rate de eski haline geri geliyor.
* combine reduce rl on plateo and CosineAnnealingLR. Or change factor with cosineannealing.

# INSTALL EXTERNAL LIBS

In [ ]:
!pip install pytorch-lightning==1.7.7
!pip install wandb==0.13.10
!pip install pycocotools==2.0.6
!pip install python-box==7.0.0
!pip install segmentation-models-pytorch==0.3.2
!pip install albumentations==1.3.0

# DEFINE SOME PARAMS

In [ ]:
cfg = {
    "max_epoch": 60,
    "max_time": "00:11:55:00", 
    "distance_transform_loss": False,
    "wandb": {
        "exp_name": "dice",
        "proj_name": "foreground-car-segm",
    },
    "model_ckpt_motior": "val_per_image_bIoU",
    "data_name": "CelebAMask-HQ-eyes",
    "output_class_num": 1,
    "mode": "binary",
    "data_dir": "/kaggle/input/celebamask-hq-v3-eyes/CelebAMask-HQ-v3-eyes/",  
    "trainer": {
        "accelerator": "auto", # gpu, tpu, cpu, auto
        "device_num": 2, # Number of gpu, check if T4 vs P100
        "precision": 16, # Double precision (64), full precision (32), half precision (16) or bfloat16 precision (bf16)
        "ckpt_path4resume": "/kaggle/input/epoch42-unet/epoch42-Unet-resnet34-lr0.0001-hight512-width512-dodspyne.ckpt",
    },
    "transform": {
        "image_resize_h": 256, # 576
        "image_resize_w": 256, # 800
    },
    "model": {
        "model_name": "Unet", # "DeepLabV3Plus", "UnetPlusPlus", "Unet"
        "encoder_name": "resnet50", # "efficientnet-b5","resnet34", "mit_b1"       
    },
    "optimizer": {
        "lr": 0.0003, # 0.001, 0.0003
        "reduce_rl_on": False,
    },   
    "patience": 20, # for validation loss
    "train_dl": {
        "batch_size": 32,
    },
    "val_dl": {
        "batch_size": 32,
    },
    "test_dl": {
        "batch_size": 32,
    },
    "SEED": 42,
    "vis_img_num": 8,
    "vis_val_batch_id": 5,
    "ckpt_save_dir": '/kaggle/working/logs/lightning_logs/checkpoints/'
}

# IMPORT SECRETS

In [ ]:
from kaggle_secrets import UserSecretsClient
secret_label = "wandb-api-key"
wandb_secret_value = UserSecretsClient().get_secret(secret_label)

# IMPORT LIBS

### Install Torch-XLA (PyTorch with Accelerated Linear Algebra (XLA) support) for TPU
#### https://www.kaggle.com/docs/tpu#tpu9
Aslında TPU VM v3-8 içerisinde torch ve torch x-la yüklü geliyor, bundan dolayı x-la yüklemeye gerek yok. Fakat TPU, pl ile stabil çalışmıyor.

In [ ]:
!python -V

In [ ]:
!nvcc --version

In [ ]:
!pip show torch

In [ ]:
!pip show torchvision

In [ ]:
!pip show torchaudio

In [ ]:
!pip show torch-xla

In [ ]:
# https://github.com/pytorch/xla#running-pytorchxla-on-cloud-tpu-and-gpu
# !pip install cloud-tpu-client==0.10 torch==1.13.0 https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/torch_xla-1.13-cp38-cp38-linux_x86_64.whl

In [ ]:
# COPIED DIRECTLY FROM LATEST PYTORCH SOURCE
import math
from torch.utils.data import Dataset
from torch import Generator
from torch import default_generator, randperm
from torch._utils import _accumulate
from torch.utils.data.dataset import Subset
from typing import (
    Generic,
    Iterable,
    Iterator,
    List,
    Optional,
    Sequence,
    Tuple,
    TypeVar,
    Union
)

T = TypeVar('T')

def random_split(dataset: Dataset[T], lengths: Sequence[Union[int, float]],
                 generator: Optional[Generator] = default_generator) -> List[Subset[T]]:
    r"""
    Randomly split a dataset into non-overlapping new datasets of given lengths.

    If a list of fractions that sum up to 1 is given,
    the lengths will be computed automatically as
    floor(frac * len(dataset)) for each fraction provided.

    After computing the lengths, if there are any remainders, 1 count will be
    distributed in round-robin fashion to the lengths
    until there are no remainders left.

    Optionally fix the generator for reproducible results, e.g.:

    >>> random_split(range(10), [3, 7], generator=torch.Generator().manual_seed(42))
    >>> random_split(range(30), [0.3, 0.3, 0.4], generator=torch.Generator(
    ...   ).manual_seed(42))

    Args:
        dataset (Dataset): Dataset to be split
        lengths (sequence): lengths or fractions of splits to be produced
        generator (Generator): Generator used for the random permutation.
    """
    if math.isclose(sum(lengths), 1) and sum(lengths) <= 1:
        subset_lengths: List[int] = []
        for i, frac in enumerate(lengths):
            if frac < 0 or frac > 1:
                raise ValueError(f"Fraction at index {i} is not between 0 and 1")
            n_items_in_split = int(
                math.floor(len(dataset) * frac)  # type: ignore[arg-type]
            )
            subset_lengths.append(n_items_in_split)
        remainder = len(dataset) - sum(subset_lengths)  # type: ignore[arg-type]
        # add 1 to all the lengths in round-robin fashion until the remainder is 0
        for i in range(remainder):
            idx_to_add_at = i % len(subset_lengths)
            subset_lengths[idx_to_add_at] += 1
        lengths = subset_lengths
        for i, length in enumerate(lengths):
            if length == 0:
                warnings.warn(f"Length of split at index {i} is 0. "
                              f"This might result in an empty dataset.")

    # Cannot verify that dataset is Sized
    if sum(lengths) != len(dataset):    # type: ignore[arg-type]
        raise ValueError("Sum of input lengths does not equal the length of the input dataset!")

    indices = randperm(sum(lengths), generator=generator).tolist()  # type: ignore[call-overload]
    return [Subset(dataset, indices[offset - length : offset]) for offset, length in zip(_accumulate(lengths), lengths)]

In [ ]:
import torch
from torch.nn import BCEWithLogitsLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CocoDetection, VisionDataset
from torchvision.utils import draw_segmentation_masks
import torchvision.transforms.functional as visF
# from torchmetrics import Dice # https://github.com/Lightning-AI/metrics/tree/master/src/torchmetrics

import os, random
from typing import Any, Callable, List, Optional, Tuple
import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image
import pandas as pd
import cv2
from pathlib import Path
from scipy import ndimage

from pprint import pprint
from box import Box
from pycocotools.coco import COCO

import pytorch_lightning as pl
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger, WandbLogger
from pytorch_lightning.utilities.model_summary import ModelSummary
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor, Callback

import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2

import wandb
wandb.login(key=wandb_secret_value)

In [ ]:
import warnings
warnings.filterwarnings("ignore") # category=DeprecationWarning

In [ ]:
cfg = Box(cfg)
pprint(cfg)

In [ ]:
def seed_everything(seed: int):  
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(cfg.SEED)

# Delete unnecessary model files from WANDB artifacts
api = wandb.Api(overrides={
        "project": cfg.wandb.proj_name
        })

artifact_type, artifact_name = "model", ... 
for version in api.artifact_versions(artifact_type, artifact_name):
    print(version.aliases)
    if len(version.aliases) == 0:
        version.delete()

# DATASET

In [ ]:
# https://pytorch.org/vision/main/_modules/torchvision/datasets/coco.html#CocoDetection
class CocoToSmpDataset(VisionDataset):
    """`MS Coco Detection <https://cocodataset.org/#detection-2016>`_ Dataset.

    It requires the `COCO API to be installed <https://github.com/pdollar/coco/tree/master/PythonAPI>`_.

    Args:
        root (string): Root directory where images are downloaded to.
        annFile (string): Path to json annotation file.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.PILToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.
    """

    def __init__(
        self,
        root: str,
        annFile: str,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        transforms: Optional[Callable] = None,
    ) -> None:
        super().__init__(root, transforms, transform, target_transform)
        from pycocotools.coco import COCO

        self.coco = COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))
        
    @staticmethod
    def _transform_binarymask_to_distance_mask(gt_mask):
        """
        Arguments:
            gt_mask (ndarray): ground-truth binary mask in HW format
        Returns:
            distance_weight (ndarray): normalized between 0 and 1 and outer of object is 1
            distance_weight_sum (float): sum af all normalized pixel values within inside of object in binary mask
        """
        # PART 1: Object distance transform
        dist = ndimage.distance_transform_cdt(gt_mask) # distance_transform_bf is less efficient
        reverse_dist = dist.max() - dist
        reverse_dist = (reverse_dist * gt_mask) # make outer background zero
        
        # CONSIDER EDGE CASES LIKE EMPTY AND FULL MASK
        if reverse_dist.max() == 0:
            if gt_mask.max() == 0: # Grount-truth mask is empty (hiç bir pixelde etiket yok)
                # every element in dist is 0
                # IoU/Distance loss is defined for non-empty classes
                distance_weight = gt_mask # full of 0s. It doesn't matter since it will be zerout out in loss func
                distance_weight_sum = np.sum(0)
                return distance_weight, distance_weight_sum
            elif gt_mask.min() == 1 : # Grount-truth mask is full (fotoğrafın her pixeli etiket)
                # every element in dist is -1
                distance_weight = gt_mask # full of 1s
                distance_weight_sum = np.sum(distance_weight)
                return distance_weight, distance_weight_sum
            else:
                # If 1s in mask is really less. Ie: small objects. Eg: gt_mask.sum() = 13
                distance_weight = gt_mask # very less 1s, too many 0s
                distance_weight_sum = np.sum(distance_weight)
                return distance_weight, distance_weight_sum
        
        inner_reverse_dist_n = reverse_dist/ reverse_dist.max() # normalize it
        # pprint(f"Inner distance transform metrics: {inner_reverse_dist_n.min()}, {inner_reverse_dist_n.max()}, {inner_reverse_dist_n.mean()}")

        # PART 2: Outer distance transform
        reverse_gt_mask = 1- gt_mask

        # PART 3: Union of inner and outer transforms
        distance_weight_vis = inner_reverse_dist_n + reverse_gt_mask # + outer_reverse_dist_n
        # plt.imshow(distance_weight_vis)
        distance_weight = inner_reverse_dist_n + reverse_gt_mask * -1 # + outer_reverse_dist_n * -1
        distance_weight_sum = np.sum(inner_reverse_dist_n)

        return distance_weight, distance_weight_sum

    def _load_image(self, id: int) -> Image.Image:
        path = self.coco.loadImgs(id)[0]["file_name"]
        img = np.array(Image.open(os.path.join(self.root, path)).convert("RGB"))
        # If you need to convert to other format HWC -> CHW: img.transpose((-1, 0, 1))
        return img
    
    def _load_mask(self, id: int) -> List[Any]:
        if not self.coco.getAnnIds(id): # check for empty masks
            # Empty mask case
            h = self.coco.loadImgs(id)[0]["height"]
            w = self.coco.loadImgs(id)[0]["width"]
            arr_2dim = np.zeros((h, w)).astype(np.float32)
            return arr_2dim # in HW format
        else:
             # Non-empty mask case
            ann = self.coco.loadAnns(self.coco.getAnnIds(id))[0]
            arr_2dim = self.coco.annToMask(ann=ann).astype(np.float32)
            return arr_2dim # in HW format

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        id = self.ids[index]
        image = self._load_image(id)
        mask = self._load_mask(id)
        
        sample = dict(image=image, mask=mask)
        if self.transforms is not None:
            sample = self.transforms(**sample)    
            sample["distance_mask"], sample["distance_mask_sum"] = self._transform_binarymask_to_distance_mask(np.array(sample["mask"])) # change binary mask to distance transformed mask
            sample["distance_mask"] = np.expand_dims(sample["distance_mask"], 0) # convert to CHW format ie. HW -> 1HW:
            sample["mask"] = np.expand_dims(sample["mask"], 0) # convert to CHW format ie. HW -> 1HW:    
        return sample
    
    def __len__(self) -> int:
        return len(self.ids)

In [ ]:
class DatasetFromSubset(Dataset):
    def __init__(self, subset, transforms=None):
        self.subset = subset
        self.transforms = transforms

    @staticmethod
    def _transform_binarymask_to_distance_mask(gt_mask):
        """
        Arguments:
            gt_mask (ndarray): ground-truth binary mask in HW format
        Returns:
            distance_weight (ndarray): normalized between 0 and 1 and outer of object is 1
            distance_weight_sum (float): sum af all normalized pixel values within inside of object in binary mask
        """
        # PART 1: Object distance transform
        dist = ndimage.distance_transform_cdt(gt_mask) # distance_transform_bf is less efficient
        reverse_dist = dist.max() - dist
        reverse_dist = (reverse_dist * gt_mask) # make outer background zero
        
        # CONSIDER EDGE CASES LIKE EMPTY AND FULL MASK
        if reverse_dist.max() == 0:
            if gt_mask.max() == 0: # Grount-truth mask is empty (hiç bir pixelde etiket yok)
                # every element in dist is 0
                # IoU/Distance loss is defined for non-empty classes
                distance_weight = gt_mask # full of 0s. It doesn't matter since it will be zerout out in loss func
                distance_weight_sum = np.sum(0)
                return distance_weight, distance_weight_sum
            elif gt_mask.min() == 1 : # Grount-truth mask is full (fotoğrafın her pixeli etiket)
                # every element in dist is -1
                distance_weight = gt_mask # full of 1s
                distance_weight_sum = np.sum(distance_weight)
                return distance_weight, distance_weight_sum
            else:
                # If 1s in mask is really less. Ie: small objects. Eg: gt_mask.sum() = 13
                distance_weight = gt_mask # very less 1s, too many 0s
                distance_weight_sum = np.sum(distance_weight)
                return distance_weight, distance_weight_sum
        
        inner_reverse_dist_n = reverse_dist/ reverse_dist.max() # normalize it
        # pprint(f"Inner distance transform metrics: {inner_reverse_dist_n.min()}, {inner_reverse_dist_n.max()}, {inner_reverse_dist_n.mean()}")

        # PART 2: Outer distance transform
        reverse_gt_mask = 1- gt_mask

        # PART 3: Union of inner and outer transforms
        distance_weight_vis = inner_reverse_dist_n + reverse_gt_mask # + outer_reverse_dist_n
        # plt.imshow(distance_weight_vis)
        distance_weight = inner_reverse_dist_n + reverse_gt_mask * -1 # + outer_reverse_dist_n * -1
        distance_weight_sum = np.sum(inner_reverse_dist_n)

        return distance_weight, distance_weight_sum
    
    def __getitem__(self, index):
        sample = self.subset[index]
        if self.transforms:
            sample = self.transforms(**sample)  

        sample["distance_mask"], sample["distance_mask_sum"] = self._transform_binarymask_to_distance_mask(np.array(sample["mask"])) # change binary mask to distance transformed mask
        sample["distance_mask"] = np.expand_dims(sample["distance_mask"], 0) # convert to CHW format ie. HW -> 1HW:        
        sample["mask"] = np.expand_dims(sample["mask"], 0) # convert to CHW format ie. HW -> 1HW:
        return sample

    def __len__(self):
        return len(self.subset)

In [ ]:
train_transform = A.Compose([
    # A.Resize(height=cfg.transform.image_resize_h, width=cfg.transform.image_resize_w),
    A.LongestMaxSize(max(cfg.transform.image_resize_h, cfg.transform.image_resize_w)),
    A.PadIfNeeded(min_height=cfg.transform.image_resize_h, min_width=cfg.transform.image_resize_w, border_mode=cv2.BORDER_REFLECT_101),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # use ImageNet image normalization 
    ToTensorV2() # numpy HWC image is converted to pytorch CHW tensor
])

val_transform = A.Compose([
    # A.Resize(height=cfg.transform.image_resize_h, width=cfg.transform.image_resize_w),
    A.LongestMaxSize(max(cfg.transform.image_resize_h, cfg.transform.image_resize_w)),
    A.PadIfNeeded(min_height=cfg.transform.image_resize_h, min_width=cfg.transform.image_resize_w, border_mode=cv2.BORDER_CONSTANT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # use ImageNet image normalization 
    ToTensorV2() # numpy HWC image is converted to pytorch CHW tensor
])

test_transform = A.Compose([
    A.LongestMaxSize(max(cfg.transform.image_resize_h, cfg.transform.image_resize_w)),
    A.PadIfNeeded(min_height=cfg.transform.image_resize_h, min_width=cfg.transform.image_resize_w, border_mode=cv2.BORDER_CONSTANT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # use ImageNet image normalization 
    ToTensorV2() # numpy HWC image is converted to pytorch CHW tensor
])

unnorm_transform = A.Compose([
    A.Normalize(mean=(-0.485/0.229, -0.456/0.224, -0.406/0.225), std=(1.0/0.229, 1.0/0.224, 1.0/0.225), max_pixel_value=1.0),
    ToTensorV2()
    ]) # -mean / std, 1.0 / std for unnormalization

# DISTANCE LOSS

In [ ]:
# https://github.com/qubvel/segmentation_models.pytorch/blob/master/segmentation_models_pytorch/losses/constants.py

#: Loss binary mode suppose you are solving binary segmentation task.
#: That mean yor have only one class which pixels are labled as **1**,
#: the rest pixels are background and labeled as **0**.
#: Target mask shape - (N, H, W), model output mask shape (N, 1, H, W).
BINARY_MODE: str = "binary"

#: Loss multiclass mode suppose you are solving multi-**class** segmentation task.
#: That mean you have *C = 1..N* classes which have unique label values,
#: classes are mutually exclusive and all pixels are labeled with theese values.
#: Target mask shape - (N, H, W), model output mask shape (N, C, H, W).
MULTICLASS_MODE: str = "multiclass"

#: Loss multilabel mode suppose you are solving multi-**label** segmentation task.
#: That mean you have *C = 1..N* classes which pixels are labeled as **1**,
#: classes are not mutually exclusive and each class have its own *channel*,
#: pixels in each channel which are not belong to class labeled as **0**.
#: Target mask shape - (N, C, H, W), model output mask shape (N, C, H, W).
MULTILABEL_MODE: str = "multilabel"

In [ ]:
# https://github.com/qubvel/segmentation_models.pytorch/blob/master/segmentation_models_pytorch/losses/_functional.py

def to_tensor(x, dtype=None) -> torch.Tensor:
    if isinstance(x, torch.Tensor):
        if dtype is not None:
            x = x.type(dtype)
        return x
    if isinstance(x, np.ndarray):
        x = torch.from_numpy(x)
        if dtype is not None:
            x = x.type(dtype)
        return x
    if isinstance(x, (list, tuple)):
        x = np.array(x)
        x = torch.from_numpy(x)
        if dtype is not None:
            x = x.type(dtype)
        return x

In [ ]:
def soft_distance_score(
    output: torch.Tensor,
    target: torch.Tensor,
    target_sum: float,
    smooth: float = 0.0,
    eps: float = 1e-7,
    dims=None,
) -> torch.Tensor:
    assert output.size() == target.size()
    if dims is not None:
        intersection = torch.sum(output * target, dim=dims)
        cardinality = torch.sum(target_sum) # torch.sum(target_sum, dim=dims)
    else:
        intersection = torch.sum(output * target)
        cardinality = torch.sum(target_sum)
        
    distance_score = intersection / (cardinality + smooth).clamp_min(eps)
    return distance_score

In [ ]:
from typing import Optional, List

import torch
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss

__all__ = ["DistanceLoss"]

class DistanceLoss(_Loss):
    def __init__(
        self,
        mode: str,
        classes: Optional[List[int]] = None,
        log_loss: bool = False,
        from_logits: bool = True,
        smooth: float = 0.0,
        eps: float = 1e-7,
    ):
        """Distance loss for image segmentation task.
        It supports binary, multiclass and multilabel cases

        Args:
            mode: Loss mode 'binary', 'multiclass' or 'multilabel'
            classes:  List of classes that contribute in loss computation. By default, all channels are included.
            log_loss: If True, loss computed as `- log(jaccard_coeff)`, otherwise `1 - jaccard_coeff`
            from_logits: If True, assumes input is raw logits
            smooth: Smoothness constant for dice coefficient
            eps: A small epsilon for numerical stability to avoid zero division error
                (denominator will be always greater or equal to eps)

        Shape
             - **y_pred** - torch.Tensor of shape (N, C, H, W)
             - **y_true** - torch.Tensor of shape (N, H, W) or (N, C, H, W)

        Reference
            # https://github.com/BloodAxe/pytorch-toolbelt
        """
        assert mode in {BINARY_MODE, MULTILABEL_MODE, MULTICLASS_MODE}
        super(DistanceLoss, self).__init__()

        self.mode = mode
        if classes is not None:
            assert mode != BINARY_MODE, "Masking classes is not supported with mode=binary"
            classes = to_tensor(classes, dtype=torch.long)

        self.classes = classes
        self.from_logits = from_logits
        self.smooth = smooth
        self.eps = eps
        self.log_loss = log_loss

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor, y_true_sum: torch.Tensor) -> torch.Tensor:

        assert y_true.size(0) == y_pred.size(0)

        if self.from_logits:
            # Apply activations to get [0..1] class probabilities
            # Using Log-Exp as this gives more numerically stable result and does not cause vanishing gradient on
            # extreme values 0 and 1
            if self.mode == MULTICLASS_MODE:
                y_pred = y_pred.log_softmax(dim=1).exp()
            else:
                y_pred = F.logsigmoid(y_pred).exp()

        bs = y_true.size(0)
        num_classes = y_pred.size(1)
        dims = (0, 2)

        if self.mode == BINARY_MODE:
            y_true = y_true.view(bs, 1, -1)
            y_pred = y_pred.view(bs, 1, -1)

        if self.mode == MULTICLASS_MODE:
            y_true = y_true.view(bs, -1)
            y_pred = y_pred.view(bs, num_classes, -1)

            y_true = F.one_hot(y_true, num_classes)  # N,H*W -> N,H*W, C
            y_true = y_true.permute(0, 2, 1)  # N, C, H*W

        if self.mode == MULTILABEL_MODE:
            y_true = y_true.view(bs, num_classes, -1)
            y_pred = y_pred.view(bs, num_classes, -1)
        
        scores = soft_distance_score(
            y_pred,
            y_true.type_as(y_pred), # y_true.type(y_pred.dtype)
            target_sum=y_true_sum,
            smooth=self.smooth,
            eps=self.eps,
            dims=dims,
        )
        if self.log_loss:
            loss = -torch.log(scores.clamp_min(self.eps))
        else:
            loss = 1.0 - scores

        # IoU loss is defined for non-empty classes
        # So we zero contribution of channel that does not have true pixels
        # NOTE: A better workaround would be to use loss term `mean(y_pred)`
        # for this case, however it will be a modified jaccard loss
        
        mask = y_true_sum.sum() > 0 # y_true.sum(dims) > 0
        loss *= mask.to(loss.dtype) # mask.float()
            
        if self.classes is not None:
            loss = loss[self.classes]

        return loss.mean()

# SCHEDULER

In [ ]:
class ReduceLROnPlateauOptimized(ReduceLROnPlateau):
    """
    Bunu implemente edince ReduceLROnPlateau ile aynı şekilde çalışıyor
    """
    def __init__(self, optimizer, **kwargs):
        super().__init__(optimizer, **kwargs)
        
    def _reduce_lr(self, epoch):
        for i, param_group in enumerate(self.optimizer.param_groups):
            old_lr = float(param_group['lr'])
            new_lr = max(old_lr * self.factor, self.min_lrs[i])
            if old_lr - new_lr > self.eps:
                param_group['lr'] = new_lr
                if self.verbose:
                    epoch_str = ("%.2f" if isinstance(epoch, float) else
                                 "%.5d") % epoch
                    print('Epoch {}: reducing learning rate'
                          ' of group {} to {:.4e}.'.format(epoch_str, i, new_lr)) 
                return new_lr
            
    def step(self, metrics, epoch=None):
        # convert `metrics` to float, in case it's a zero-dim Tensor
        current = float(metrics)
        if epoch is None:
            epoch = self.last_epoch + 1
        else:
            warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
        self.last_epoch = epoch

        if self.is_better(current, self.best):
            self.best = current
            self.num_bad_epochs = 0
        else:
            self.num_bad_epochs += 1

        if self.in_cooldown:
            self.cooldown_counter -= 1
            self.num_bad_epochs = 0  # ignore any bad epochs in cooldown

        if self.num_bad_epochs > self.patience:
            new_lr = self._reduce_lr(epoch)
            self.cooldown_counter = self.cooldown
            self.num_bad_epochs = 0

        self._last_lr = [group['lr'] for group in self.optimizer.param_groups]
        
        if 'new_lr' in locals():
            return new_lr

# BOUNDARY IOU TENSOR IMPLEMENTATION

In [ ]:
# General util function to get the boundary of a binary mask.
def mask_to_boundary_tensor(mask, dilation_ratio=0.02):
    """
    Convert binary mask to boundary mask.
    Arguments:
        mask (torch.Tensor): mask of torch.Tensor of shape (N, C, H, W)
        dilation_ratio (float): ratio to calculate dilation = dilation_ratio * image_diagonal
    Returns: 
        boundary mask (torch.Tensor): boundary mask of torch.Tensor of shape (N, C, H, W)
    """
    nb, c, h, w = mask.shape
    img_diag = np.sqrt(h ** 2 + w ** 2)
    dilation = int(round(dilation_ratio * img_diag))
    if dilation < 1:
        dilation = 1
        
    boundary_mask = torch.zeros_like(mask)
    kernel = np.ones((3, 3), dtype=np.uint8)
    for idx, img_tensor in enumerate(mask[:,]): # tensor_img will be in the shape of (C, H, W)
        # From tensor to numpy
        # .cpu().detach().squeeze().numpy()
        img_np = np.array(img_tensor.cpu().detach(), dtype="uint8").transpose(1, 2, 0).squeeze() # in the shape of (H, W, C) -> (H, W) since C=1
        # Pad image so mask truncated by the image border is also considered as boundary.
        new_mask = cv2.copyMakeBorder(img_np, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)
        new_mask_erode = cv2.erode(new_mask, kernel, iterations=dilation)
        mask_erode = new_mask_erode[1 : h + 1, 1 : w + 1] # in the shape of (H, W)
        boundary_mask_np = img_np - mask_erode
        boundary_mask_np = np.expand_dims(boundary_mask_np, 0)
        # import pdb; pdb.set_trace()
        # From numpy to tensor
        boundary_mask[idx,] = torch.tensor(boundary_mask_np)
    return boundary_mask

# MODEL

This is just regular PyTorch organized in a specific format.

 Notice the following:

* no GPU/TPU specific code
* no .to(device)
* np .cuda()

In [ ]:
# https://pytorch-lightning.readthedocs.io/en/stable/guides/speed.html
class ImageSegModel(pl.LightningModule):
    def __init__(self, cfg, train_transform, val_transform, test_transform, unnorm_transform):
        '''method used to define our model and data parameters'''
        super().__init__()    
        # Set our init args as class attributes
        self.n_cpu = os.cpu_count()
        self.cfg = cfg
        
        # Define PyTorch model, model weights are in cuda
        encoder_weights = "imagenet"
        input_channels = 3
        self.model = smp.create_model(arch=cfg.model.model_name,
                                      encoder_name=cfg.model.encoder_name,
                                      encoder_weights=encoder_weights,
                                      in_channels=input_channels,
                                      classes=cfg.output_class_num)
        
        # Preprocessing/Transformations  
        self.train_transform = train_transform
        self.val_transform = val_transform
        self.test_transform = test_transform
        self.unnorm_transform = unnorm_transform
        
        # Define loss. If predicted mask contains logits, and loss param `from_logits` is set to True
        if cfg.distance_transform_loss:
            self.loss_distance = DistanceLoss(BINARY_MODE, from_logits=True)
        else:
            self.loss_dice = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
            # self.loss_bce = BCEWithLogitsLoss()
            # self.loss_dice = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
            # self.loss_focal = smp.losses.FocalLoss(smp.losses.BINARY_MODE) # it uses focal_loss_with_logits
            # self.loss_iou = smp.losses.JaccardLoss(smp.losses.BINARY_MODE, from_logits=False)
        self.distance_transform_metric = DistanceLoss(BINARY_MODE, from_logits=False)
        
        self.save_hyperparameters(ignore=["model"])
    
    def forward(self, x):
        '''method used for inference input -> output'''
        return self.model(x) # logits

    def _shared_step(self, batch, stage):
        '''needs to return a loss from a single batch'''
        x, y, y_distance, y_distance_sum = batch["image"], batch["mask"], batch["distance_mask"], batch["distance_mask_sum"]
            
        # Shape of the image should be [batch_size, num_channels, height, width]
        # if you work with grayscale images, expand channels dim to have [batch_size, 1, height, width]
        assert x.ndim == 4
    
        # Shape of the mask should be [batch_size, num_classes, height, width]. For binary segmentation num_classes=1
        assert y.ndim == 4
        assert y_distance.ndim == 4
        
        # Check that image dimensions are divisible by 32, encoder and decoder connected by `skip connections`.
        # Usually encoder have 5 stages of downsampling by factor 2 (2 ^ 5 = 32); e.g. if we have image with shape 65x65 we will have 
        # following shapes of features in encoder and decoder: 84, 42, 21, 10, 5 -> 5, 10, 20, 40, 80 and we will get an error trying to concat these features
        h, w = x.shape[2:]
        assert h % 32 == 0 and w % 32 == 0
        
        # Check that mask values in between 0 and 1, NOT 0 and 255 for binary segmentation
        assert y.max() <= 1.0 and y.min() >= 0
        assert y_distance.max() <= 1.0 and y_distance.min() >= -1    
        
        logits_y = self(x) # it is actually self.forward(x)
        # Lets compute metrics for some threshold. First convert mask values to probabilities, then apply thresholding
        prob_y = logits_y.sigmoid()
        pred_y_th = (prob_y > 0.95).float()
        
        boundary_y = mask_to_boundary_tensor(y, dilation_ratio=0.02)
        boundary_pred_y = mask_to_boundary_tensor(prob_y, dilation_ratio=0.02)
        
        if cfg.distance_transform_loss:
            loss = self.loss_distance(logits_y, y_distance, y_distance_sum)
        else:
            loss_dice = self.loss_dice(logits_y, y)
            # loss_bce = self.loss_bce(logits_y, y)
            loss = loss_dice
            # loss = loss_dice + loss_bce
            # loss_dice = self.loss_dice(logits_y, y)
            # loss_focal = self.loss_focal(logits_y, y)
            # loss = loss_dice + loss_focal
            # if self.current_epoch > -1:
            #     loss_iou = self.loss_iou(prob_y, y)
            #     loss_biou = self.loss_iou(boundary_pred_y, boundary_y)
            #     loss = torch.maximum(loss_iou, loss_biou)
            # else:
            #     loss = self.loss_iou(prob_y, y)
        self.log(f"{stage}_loss", loss, on_step=False, on_epoch=True, prog_bar=True)

        # We will compute IoU metric by two ways
        #   1. dataset-wise
        #   2. image-wise
        # but for now we just compute true positive, false positive, false negative and true negative 'pixels' for each image and class
        # these values will be aggregated in the end of an epoch
        tp, fp, fn, tn = smp.metrics.get_stats(pred_y_th.long(), y.long(), mode=self.cfg.mode, num_classes=self.cfg.output_class_num)
        
        # Extract tp, fp, fn, tn for boundary iou 
        boundary_pred_y_th = mask_to_boundary_tensor(pred_y_th, dilation_ratio=0.02)
        b_tp, b_fp, b_fn, b_tn = smp.metrics.get_stats(boundary_pred_y_th.long(), boundary_y.long(), mode="binary")
        
        # Calculate distance transform evaluation metric here. Do not concaterate in the epoch_end and then evaluate since it will require more computation.
        distance_transform_metric = 1- self.distance_transform_metric(pred_y_th, y_distance, y_distance_sum)
        self.log(f"{stage}_distance_transform_evalmetric", distance_transform_metric, on_step=False, on_epoch=True, prog_bar=True)
        
        return {
            "loss": loss,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn,
            "b_tp": b_tp,
            "b_fp": b_fp,
            "b_fn": b_fn,
            "b_tn": b_tn,
            "prob_y": prob_y
        }
     
    def training_step(self, batch, batch_idx):
        return self._shared_step(batch, "train")
 
    def validation_step(self, batch, batch_idx):
        return self._shared_step(batch, "val")
    
    def test_step(self, batch, batch_idx):
        return self._shared_step(batch, "test")

    def _shared_step_end(self, batch_parts):
        # Check if there is multi-gpu or not. This step must be here due to pl lib.
        if batch_parts["loss"].numel() != 1: # multi-gpu case
            batch_parts["loss"] = torch.mean(batch_parts["loss"], dim=0) # take the mean of losses from different GPUs
        return batch_parts
    
    def training_step_end(self, batch_parts):
        return self._shared_step_end(batch_parts) 

    def validation_step_end(self, batch_parts):
        return self._shared_step_end(batch_parts) 
    
    def test_step_end(self, batch_parts):
        return self._shared_step_end(batch_parts) 

    def _shared_epoch_end(self, step_outputs, stage):
        # outputs are coming from the result or trainin/validation/test steps respectively
        # aggregate step metrics
        tp = torch.cat([x["tp"] for x in step_outputs])
        fp = torch.cat([x["fp"] for x in step_outputs])
        fn = torch.cat([x["fn"] for x in step_outputs])
        tn = torch.cat([x["tn"] for x in step_outputs])
        b_tp = torch.cat([x["b_tp"] for x in step_outputs])
        b_fp = torch.cat([x["b_fp"] for x in step_outputs])
        b_fn = torch.cat([x["b_fn"] for x in step_outputs])
        b_tn = torch.cat([x["b_tn"] for x in step_outputs])

        # per image IoU means that we first calculate IoU score for each image and then compute mean over these scores
        per_image_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro-imagewise")
        
        # dataset IoU means that we aggregate intersection and union over whole dataset and then compute IoU score. 
        # The difference between dataset_iou and per_image_iou scores in this particular case will not be much, 
        # however for dataset with "empty" images (images without target class) a large gap could be observed. 
        # Empty images influence a lot on per_image_iou and much less on dataset_iou.
        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        
        per_image_dice = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro-imagewise")
        dataset_dice = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro")
        
        per_image_fbeta = smp.metrics.fbeta_score(tp, fp, fn, tn, reduction="micro-imagewise")
        dataset_fbeta = smp.metrics.fbeta_score(tp, fp, fn, tn, reduction="micro")
        
        per_image_sensitivity= smp.metrics.sensitivity(tp, fp, fn, tn, reduction="micro-imagewise")
        dataset_sensitivity = smp.metrics.sensitivity(tp, fp, fn, tn, reduction="micro")
        
        per_image_specificity = smp.metrics.specificity(tp, fp, fn, tn, reduction="micro-imagewise")
        dataset_specificity = smp.metrics.specificity(tp, fp, fn, tn, reduction="micro")
        
        per_image_bIoU = smp.metrics.iou_score(b_tp, b_fp, b_fn, b_tn, reduction="micro-imagewise")
        dataset_bIoU = smp.metrics.iou_score(b_tp, b_fp, b_fn, b_tn, reduction="micro")
        
        metrics = {
            f"{stage}_per_image_iou": per_image_iou,
            f"{stage}_dataset_iou": dataset_iou,
            f"{stage}_per_image_dice": per_image_dice,
            f"{stage}_dataset_dice": dataset_dice,
            f"{stage}_per_image_fbeta": per_image_fbeta,
            f"{stage}_dataset_fbeta": dataset_fbeta,
            f"{stage}_per_image_sensitivity": per_image_sensitivity,
            f"{stage}_dataset_sensitivity": dataset_sensitivity,
            f"{stage}_per_image_specificity": per_image_specificity,
            f"{stage}_dataset_specificity": dataset_specificity,
            f"{stage}_per_image_bIoU": per_image_bIoU,
            f"{stage}_dataset_bIoU": dataset_bIoU,
        }
        
        self.log_dict(metrics, prog_bar=True)
    
    def training_epoch_end(self, training_step_outputs):
        return self._shared_epoch_end(training_step_outputs, "train")

    def validation_epoch_end(self, validation_step_outputs):
        return self._shared_epoch_end(validation_step_outputs, "val")

    def test_epoch_end(self, test_step_outputs):
        return self._shared_epoch_end(test_step_outputs, "test")  
            
    def configure_optimizers(self):
        '''defines model optimizer and scheduler'''
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.cfg.optimizer.lr)
        if self.cfg.optimizer.reduce_rl_on:          
            # It is called automatic optimization if `scheduler.step()` is not called manually inside pl.LightningModule. Otherwise, it is manuel optimization
            scheduler = ReduceLROnPlateau(optimizer, mode="min", patience=5, factor=0.5, min_lr=1e-7) # ReduceLROnPlateauOptimized
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "interval": "epoch", # The unit of the scheduler's step size, could also be 'step'. 'epoch' updates the scheduler on epoch end whereas 'step' updates it after a optimizer update.
                    "monitor": "val_loss", # Metric to monitor for schedulers like ReduceLROnPlateau
                    "frequency": 1 # How many epochs/steps should pass between calls to `scheduler.step()`. 1 corresponds to updating the learning rate after every epoch/step. 
                    # If "monitor" references validation metrics, then "frequency" should be set to a multiple of "trainer.check_val_every_n_epoch".
                    # "frequency" and "interval" will be ignored even if they are provided in here configure_optimizers() during manual optimization
                },
            }
        else:
            return optimizer
    ####################
    # DATA RELATED HOOKS
    ####################

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            full_ds = CocoToSmpDataset(root=os.path.join(self.cfg.data_dir, "train"), 
                                       annFile=os.path.join(self.cfg.data_dir, "annotations_train.json")
                                       )
            # Train-val split before appliying transformations
            train_subset, val_subset = random_split(full_ds, [0.8, 0.2],
                                                      generator=torch.Generator().manual_seed(self.cfg.SEED))
            # Apply transformations to each subset
            self.train_ds = DatasetFromSubset(train_subset, transforms=self.train_transform)
            self.val_ds = DatasetFromSubset(val_subset, transforms=self.val_transform)
        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.test_ds = CocoToSmpDataset(root=os.path.join(self.cfg.data_dir, "test"), 
                                            annFile=os.path.join(self.cfg.data_dir, "annotations_test.json"),
                                            transforms=self.test_transform)
            
    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.cfg.train_dl.batch_size, shuffle=True, num_workers=self.n_cpu, pin_memory=True)
        # pin_memory will put the fetched data Tensors in pinned memory, and thus enables faster data transfer to CUDA-enabled GPUs

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.cfg.val_dl.batch_size, shuffle=False, num_workers=self.n_cpu, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.cfg.test_dl.batch_size, shuffle=False, num_workers=self.n_cpu, pin_memory=True)

In [ ]:
# https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html#on-validation-epoch-end
class ReduceLROnPlateauOptCallback(Callback): 
    def on_validation_epoch_end(self, trainer, pl_module):
        """Called when the validation epoch ends."""
        # Manual optimization. Note that step should be called after val_loss is calculated
        sch = pl_module.lr_schedulers()
        # If the selected scheduler is a ReduceLROnPlateau scheduler.
        if isinstance(sch, ReduceLROnPlateau):
            new_lr = sch.step(pl_module.trainer.callback_metrics["val_loss"])
            if new_lr is None:
                print("Regular ReduceLROnPlateau")
            elif isinstance(new_lr, float):
                print(f"Optimized ReduceLROnPlateau with new lr of {new_lr}")
                trainer.fit(pl_module, ckpt_path="best")

In [ ]:
# https://docs.wandb.ai/guides/integrations/lightning
# to control when you log to Weights & Biases via the WandbLogger
class LogSegPredictionCallback(Callback): 
    def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
        """Called when the validation batch ends."""
        # outputs: The outputs of validation_step_end(validation_step(x))
        # Let's log all sample image predictions from the fifth batch
        if batch_idx == pl_module.cfg.vis_val_batch_id: 
            with torch.no_grad():   
                # x, y = batch["image"], batch["mask"]
                x, y, y_distance, y_distance_sum = batch["image"], batch["mask"], batch["distance_mask"], batch["distance_mask_sum"]
                
                # All five images and preds in one figure
                fig1, axs1 = plt.subplots(pl_module.cfg.vis_img_num, 5, figsize=(50, 50)) # pl_module.cfg.val_dl.batch_size
                for img_id, (img_gt, mask_gt, mask_dist_gt, mask_pred) in enumerate(zip(x, y, y_distance, outputs["prob_y"])): # [1, num_channels, height, width]
                    mask_pred_th = (mask_pred > 0.5).float()
                    un_img = img_gt.cpu().numpy().squeeze().transpose(1, 2, 0) # for visualization we have to transpose back to HWC
                    img_gt = (pl_module.unnorm_transform(image=un_img)["image"]*255).to(torch.uint8)
                    
                    axs1[img_id, 0].imshow(np.asarray(visF.to_pil_image(img_gt)))
                    axs1[img_id, 0].set_title("Image", fontsize=30)

                    axs1[img_id, 1].imshow(mask_dist_gt.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs1[img_id, 1].set_title("GT-DistanceMask", fontsize=30)
                    
                    axs1[img_id, 2].imshow(mask_gt.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs1[img_id, 2].set_title("GT-Mask", fontsize=30)
    
                    axs1[img_id, 3].imshow(mask_pred.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs1[img_id, 3].set_title("Pred-Mask", fontsize=30)
                        
                    axs1[img_id, 4].imshow(mask_pred_th.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs1[img_id, 4].set_title("Pred-Mask Thresholded", fontsize=30)
                    
                    # Save only first 4 images in the batch
                    if img_id == pl_module.cfg.vis_img_num - 1:
                        break
                plt.show()
                img_dir = os.path.join("/kaggle/working/", f"epoch{pl_module.current_epoch}_data_pred_vis.png")
                fig1.savefig(img_dir, bbox_inches='tight') 
                wandb_logger.log_image(key=f"val_batch{batch_idx}_all", images=[img_dir])
                
                # Four images and their preds are in different figures
                fig2, axs2 = plt.subplots(2, 2, figsize=(30, 30)) 
                for img_id, (img_gt, mask_gt, mask_pred) in enumerate(zip(x, y, outputs["prob_y"])): # [1, num_channels, height, width]
                    mask_pred_th = (mask_pred > 0.5).squeeze(0)
                    mask_gt_th = mask_gt > 0.5 # boolean
                    
                    un_img = img_gt.cpu().numpy().squeeze().transpose(1, 2, 0) # for visualization we have to transpose back to HWC
                    img = (pl_module.unnorm_transform(image=un_img)["image"]*255).to(torch.uint8)
                         
                    gt_mask_on_img = draw_segmentation_masks(image=img, masks=mask_gt_th, alpha=0.7, colors=["orange"]) # Tensor[C, H, W]
                    pred_mask_on_img = draw_segmentation_masks(image=img, masks=mask_pred_th, alpha=0.7, colors=["orange"]) # Tensor[C, H, W]
                    
                    axs2[0, 0].imshow(np.asarray(visF.to_pil_image(gt_mask_on_img)))
                    axs2[0, 0].set_title("GT-Mask on Image", fontsize=30)
                    
                    axs2[0, 1].imshow(np.asarray(visF.to_pil_image(pred_mask_on_img)))
                    axs2[0, 1].set_title("Pred-Mask on Image", fontsize=30)   
        
                    axs2[1, 0].imshow(mask_gt.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs2[1, 0].set_title("GT-Mask", fontsize=30)
                        
                    axs2[1, 1].imshow(mask_pred.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs2[1, 1].set_title("Pred-Mask Without Threshold", fontsize=30)
                    
                    plt.show()
                    
                    img_dir = os.path.join("/kaggle/working/", f"epoch{pl_module.current_epoch}_val_batch{batch_idx}_image{img_id}.png")
                    fig2.savefig(img_dir, bbox_inches='tight') 
                    wandb_logger.log_image(key=f"val_batch{batch_idx}_image{img_id}", images=[img_dir])
                    
                    # Save only first 4 images in the batch
                    if img_id == pl_module.cfg.vis_img_num - 1:
                        break
                        
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        """Called when the train batch ends."""
        # outputs: The outputs of train_step_end(train_step(x))??
        # Let's log all sample image predictions from the fifth batch
        if batch_idx == pl_module.cfg.vis_val_batch_id: 
            with torch.no_grad():   
                # x, y = batch["image"], batch["mask"]
                x, y, y_distance, y_distance_sum = batch["image"], batch["mask"], batch["distance_mask"], batch["distance_mask_sum"]
                
                # All five images and preds in one figure
                fig1, axs1 = plt.subplots(pl_module.cfg.vis_img_num, 5, figsize=(50, 50)) # pl_module.cfg.val_dl.batch_size
                for img_id, (img_gt, mask_gt, mask_dist_gt, mask_pred) in enumerate(zip(x, y, y_distance, outputs["prob_y"])): # [1, num_channels, height, width]
                    mask_pred_th = (mask_pred > 0.5).float()
                    un_img = img_gt.cpu().numpy().squeeze().transpose(1, 2, 0) # for visualization we have to transpose back to HWC
                    img_gt = (pl_module.unnorm_transform(image=un_img)["image"]*255).to(torch.uint8)
                    
                    axs1[img_id, 0].imshow(np.asarray(visF.to_pil_image(img_gt)))
                    axs1[img_id, 0].set_title("Image", fontsize=30)

                    axs1[img_id, 1].imshow(mask_dist_gt.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs1[img_id, 1].set_title("GT-DistanceMask", fontsize=30)
                    
                    axs1[img_id, 2].imshow(mask_gt.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs1[img_id, 2].set_title("GT-Mask", fontsize=30)
    
                    axs1[img_id, 3].imshow(mask_pred.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs1[img_id, 3].set_title("Pred-Mask", fontsize=30)
                        
                    axs1[img_id, 4].imshow(mask_pred_th.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs1[img_id, 4].set_title("Pred-Mask Thresholded", fontsize=30)
                    
                    # Save only first 4 images in the batch
                    if img_id == pl_module.cfg.vis_img_num - 1:
                        break
                plt.show()
                img_dir = os.path.join("/kaggle/working/", f"train_epoch{pl_module.current_epoch}_data_pred_vis.png")
                fig1.savefig(img_dir, bbox_inches='tight') 
                wandb_logger.log_image(key=f"train_batch{batch_idx}_all", images=[img_dir])
                
                # Four images and their preds are in different figures
                fig2, axs2 = plt.subplots(2, 2, figsize=(30, 30)) 
                for img_id, (img_gt, mask_gt, mask_pred) in enumerate(zip(x, y, outputs["prob_y"])): # [1, num_channels, height, width]
                    mask_pred_th = (mask_pred > 0.5).squeeze(0)
                    mask_gt_th = mask_gt > 0.5 # boolean
                    
                    un_img = img_gt.cpu().numpy().squeeze().transpose(1, 2, 0) # for visualization we have to transpose back to HWC
                    img = (pl_module.unnorm_transform(image=un_img)["image"]*255).to(torch.uint8)
                         
                    gt_mask_on_img = draw_segmentation_masks(image=img, masks=mask_gt_th, alpha=0.7, colors=["orange"]) # Tensor[C, H, W]
                    pred_mask_on_img = draw_segmentation_masks(image=img, masks=mask_pred_th, alpha=0.7, colors=["orange"]) # Tensor[C, H, W]
                    
                    axs2[0, 0].imshow(np.asarray(visF.to_pil_image(gt_mask_on_img)))
                    axs2[0, 0].set_title("GT-Mask on Image", fontsize=30)
                    
                    axs2[0, 1].imshow(np.asarray(visF.to_pil_image(pred_mask_on_img)))
                    axs2[0, 1].set_title("Pred-Mask on Image", fontsize=30)   
        
                    axs2[1, 0].imshow(mask_gt.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs2[1, 0].set_title("GT-Mask", fontsize=30)
                        
                    axs2[1, 1].imshow(mask_pred.cpu().numpy().squeeze())  # for visualization we have to remove 3rd dimension of mask
                    axs2[1, 1].set_title("Pred-Mask Without Threshold", fontsize=30)
                    
                    plt.show()
                    
                    img_dir = os.path.join("/kaggle/working/", f"epoch{pl_module.current_epoch}_train_batch{batch_idx}_image{img_id}.png")
                    fig2.savefig(img_dir, bbox_inches='tight') 
                    wandb_logger.log_image(key=f"train_batch{batch_idx}_image{img_id}", images=[img_dir])
                    
                    # Save only first 4 images in the batch
                    if img_id == pl_module.cfg.vis_img_num - 1:
                        break

In [ ]:
# WANDB SEGMENTASYON GÖRSELLEŞTİRMELERİ PEK İYİ DEĞİL
# https://docs.wandb.ai/guides/track/log/media#image-overlays
# to control when you log to Weights & Biases via the WandbLogger
class LogSegPredOverlayCallback(Callback):
    def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
        """Called when the validation batch ends."""
        # outputs: The outputs of validation_step_end(validation_step(x))
        # Let's log all sample image predictions from the fifth batch
        if batch_idx == pl_module.cfg.vis_val_batch_id:
            # self.eval()
            with torch.no_grad():   
                x, y = batch["image"], batch["mask"]
                images = [img for img in x]
                # for visualization in wandb, masks must be in numpy format without 3rd dimension
                masks = [{"predictions": {"mask_data": (mask_pred > 0.5).float().cpu().detach().numpy().squeeze(), "class_labels": {1: "car"}},
                          "ground_truth": {"mask_data": mask_gt.cpu().detach().numpy().squeeze(), "class_labels": {1: "car"}}}
                          for mask_gt, mask_pred in zip(y, outputs["prob_y"])]
            wandb_logger.log_image(
                key=f"val_batch{batch_idx}_image{img_id}", 
                images=images, 
                masks=masks)

In [ ]:
model = ImageSegModel(cfg, train_transform, val_transform, test_transform, unnorm_transform)
ModelSummary(model) #to see detailed layer based parameter nums max_depth=-1

In [ ]:
wandb_logger = WandbLogger(
    project=cfg.wandb.proj_name,
    name=f"{cfg.model.model_name}/{cfg.model.encoder_name}/{cfg.wandb.exp_name}",
    group= f"{cfg.model.model_name}/{cfg.model.encoder_name}/{cfg.data_name}",
    log_model="all", # model checkpoints are logged during training
)
wandb_logger.watch(model, log="all") # log and monitor gradients, parameter histogram and model topology as we train  
# W&B summary metric to display the min, max, mean or best value for that metric
wandb.define_metric('train_per_image_dice', summary='max')
wandb.define_metric('val_per_image_dice', summary='max')
wandb.define_metric('train_per_image_iou', summary='max')
wandb.define_metric('val_per_image_iou', summary='max')
wandb.define_metric('train_per_image_bIoU', summary='max')
wandb.define_metric('val_per_image_bIoU', summary='max')
wandb.define_metric('train_distance_transform_evalmetric', summary='max')
wandb.define_metric('val_distance_transform_evalmetric', summary='max')
wandb.define_metric('train_loss', summary='min')
wandb.define_metric('val_loss', summary='min')

model_checkpointer = ModelCheckpoint(
    monitor=cfg.model_ckpt_motior,
    mode="max", # log model only if `val_per_image_iou` increases
    save_top_k=1, # to save the best model. save_top_k=-1 to save all models
    # every_n_epochs=5, # to save at every n epochs
    save_last=True,
    # To save locally:
    dirpath=cfg.ckpt_save_dir,
    filename='{epoch}-'+f'{cfg.model.model_name}-{cfg.model.encoder_name}-lr{cfg.optimizer.lr}-hight{cfg.transform.image_resize_h}-width{cfg.transform.image_resize_w}-{cfg.data_name}'
)

earlystop_checkpointer = EarlyStopping(
    monitor="val_loss", mode="min", patience=cfg.patience, verbose=True
) # verbose = 0, means silent.

lr_monitor = LearningRateMonitor() # logging_interval='epoch'/'step'. Set to None to log at individual interval according to the interval key of each scheduler

In [ ]:
trainer = pl.Trainer(
    max_epochs=cfg.max_epoch,
    max_time=cfg.max_time, # Stop after max_time hours of training or when reaching max_epochs epochs
    precision=cfg.trainer.precision, # Mixed Precision (16-bit) combines the use of both 32 and 16-bit floating points to reduce memory footprint
    accelerator=cfg.trainer.accelerator, # gpu, tpu, auto
    devices=cfg.trainer.device_num,
    callbacks=[TQDMProgressBar(refresh_rate=20), # for notebook usage
               # earlystop_checkpointer,
               model_checkpointer,
               LogSegPredictionCallback(),
               lr_monitor,
               # ReduceLROnPlateauOptCallback()
              ],
    logger=[CSVLogger(save_dir="logs/"), wandb_logger], # multiple loggers
    strategy=None if cfg.trainer.device_num==1 else 'dp', # 'dp' strategy is used when multiple-gpus with 1 machine.
    deterministic=True, # for reproducibity
)

In [ ]:
trainer.fit(model) # ckpt_path=cfg.trainer.ckpt_path4resume, ckpt_path="best"

In [ ]:
trainer.test(model, ckpt_path="best") # ckpt_path="last" to load and evaluate the last model
# EXPERIMENT 1
# unet with resnet34 encoder
    # 1 T4 GPU: 8.9 dk + 8.50 dk-> 2 T4 GPU: 7.22 dk + 6.48 dk

# EXPERIMENT 2
# unet with resnet34 encoder
    # 2 T4 GPU & num_workers=0: 7.38 dk + 7.2 dk -> 2 T4 GPU & num_workers=2: 6.31 dk + 6.32 dk ---
    # ---> 2 T4 GPU & num_workers=2 & mixed precision & 512x512: 4.1 dk + 4.1 dk
    
# EXPERIMENT 3
# unet with resnet34 encoder
    # 2 T4 GPU & num_workers=2 & mixed precision & 512x512 & wandb & pin_memory: 4.20 dk + 4.20 dk ---
    # ---> 1 P100 GPU & num_workers=2 & mixed precision & 512x512 & wandb & pin_memory: 7.03 dk + 7.03 dk
    
# EXPERIMENT 4
# unet with resnet34 encoder
    # 2 T4 GPU & num_workers=2 & mixed precision & 512x512 & wandb & pin_memory & batch size 4: 4.28 dk + 4.28 dk ---
    # ---> 2 T4 GPU & num_workers=2 & mixed precision & 512x512 & wandb & pin_memory & batch size 16: 2.39 dk + 3.01 dk + 2.57 dk
    # ---> 2 T4 GPU & num_workers=2 & mixed precision & 512x512 & wandb & pin_memory & batch size 64: 2.47 dk + 2.46 dk
    # ---> 2 T4 GPU & num_workers=2 & mixed precision & 512x512 & wandb & pin_memory & batch size 128: NOT FITTING INTO VRAM
    
# EXPERIMENT 5
# unet with resnet34 encoder
    # 2 T4 GPU & num_workers=2 & mixed precision & 512x512 & wandb & pin_memory & batch size 64 & dice and bce loss: 2.47 dk + 2.46 dk
    # 2 T4 GPU & num_workers=2 & mixed precision & 512x512 & wandb & pin_memory & batch size 64 & distance loss: 3.34 dk + 3.39 dk

In [ ]:
wandb.finish()

# LOAD MODEL FROM CKPT

In [ ]:
model4eval = model # ImageSegModel.load_from_checkpoint("/kaggle/input/unet-sil/model_deneme")

# disable randomness, dropout, etc...
model4eval.eval()

In [ ]:
test_ds = CocoToSmpDataset(root=os.path.join(cfg.data_dir, "test"), 
                                annFile=os.path.join(cfg.data_dir, "annotations_test.json"),
                                transforms=test_transform
                          )

In [ ]:
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False, figsize=(50, 50))
    for i, img in enumerate(imgs):
        img = img.detach()
        img = visF.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
        return fig, axs

In [ ]:
un_img = np.array(test_ds[4]["image"]).transpose((1, 2, 0)) # CHW -> HWC
img = (unnorm_transform(image=un_img)["image"]*255).to(torch.uint8)

In [ ]:
mask_gt = torch.tensor(test_ds[4]["mask"]) > 0.5 # boolean
show(draw_segmentation_masks(image=img, masks=mask_gt, alpha=0.5, colors=["orange"]))

In [ ]:
# predict with the model
un_img = test_ds[4]["image"].unsqueeze(0)
y_hat = model4eval(un_img)
mask_pred = y_hat.sigmoid()
mask_pred = (mask_pred > 0.5).squeeze(0)
show(draw_segmentation_masks(image=img, masks=mask_pred, alpha=0.5, colors=["orange"]))

# LOAD MODEL FROM WANDB CKPT

In [ ]:
# run = wandb.init(project="foreground-car-segm")

# reference can be retrieved in artifacts panel
# "VERSION" can be a version (ex: "v2") or an alias ("latest or "best")
# checkpoint_reference = "frkangul/foreground-car-segm/model-ugk2wlcc:v7"

# download checkpoint locally (if not already cached)
# artifact = run.use_artifact(checkpoint_reference, type="model")
# artifact_dir = artifact.download()

# load checkpoint
# model4eval = ImageSegModel.load_from_checkpoint(Path(artifact_dir) / "model.ckpt")

In [ ]:
# trainer.test(model4eval)